In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter
import nltk

import warnings

nltk.download('gutenberg')
!python -m spacy download en

[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/rodrickleary/nltk_data...
[nltk_data]   Unzipping corpora/gutenberg.zip.


     |████████████████████████████████| 12.0MB 4.0MB/s eta 0:00:01    |██████████████████████████      | 9.7MB 4.0MB/s eta 0:00:01
  Created wheel for en-core-web-sm: filename=en_core_web_sm-2.2.5-cp37-none-any.whl size=12011740 sha256=f7b03cee5461f46ee04777fd50cd53eed014dbc6d8285c2deabec2410307688b
  Stored in directory: /private/var/folders/79/qhbcx54131gfsh0zc616f06c0000gn/T/pip-ephem-wheel-cache-lydaebtm/wheels/6a/47/fb/6b5a0b8906d8e8779246c67d4658fd8a544d4a03a75520197a
Successfully built en-core-web-sm
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/opt/anaconda3/lib/python3.7/site-packages/en_core_web_sm -->
/opt/anaconda3/lib/python3.7/site-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
# supress future warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
# Utility function for standard text cleaning.
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text
    
# Load and clean the data.
persuasion = gutenberg.raw('austen-persuasion.txt')
alice = gutenberg.raw('carroll-alice.txt')

# The Chapter indicator is idiosyncratic
persuasion = re.sub(r'Chapter \d+', '', persuasion)
alice = re.sub(r'CHAPTER .*', '', alice)
    
alice = text_cleaner(alice[:int(len(alice)/10)])
persuasion = text_cleaner(persuasion[:int(len(persuasion)/10)])

In [4]:
# Parse the cleaned novels. This can take a bit.
nlp = spacy.load('en')
alice_doc = nlp(alice)
persuasion_doc = nlp(persuasion)

In [5]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(2000)]

In [6]:
import collections
# define a new bag of words approach that should get us some additional information
# we want to look at number of words in sentence, how many times puncutation is used, how many nouns, verbs, adj, etc. length of previous sentence vs. length of next sentence
def bow_feature_generator(sentences, common_words, include_extra_counts=True):
    # we want to go through each sentence and count the number of occurances of verbs, nouns, adj, etc.
    # we also want to count how many words are in each text
    rows = []
    for index, row in enumerate(sentences.itertuples()):
        
        sentence = row.text_sentence
        source = row.text_source
        
        info_row = collections.defaultdict(int)
        
        for token in sentence:
            if include_extra_counts:
                part_of_speech = token.pos_

                if part_of_speech == "VERB":
                    # if it is a verb, add 1 for the sentence
                    info_row["n_verbs"] += 1
                elif part_of_speech == "NOUN":
                    # if it is a noun, add 1 for the noun to the sentence
                    info_row["n_nouns"] += 1

                elif part_of_speech == "ADJ":
                    # if it is an adjective, add 1 for the adjective count for this sentence
                    info_row["n_adjectives"] += 1

                info_row["n_words"] += 1
                
                
                if token.is_punct:
                    info_row["n_puncutated_words"] += 1
                    
                if token.is_right_punct:
                    info_row["n_right_puncutated_words"] += 1
                
                if token.is_left_punct:
                    info_row["n_left_puncutated_words"] += 1
                
                # named entities
                if token.ent_type_ == "PERSON":
                    info_row["n_people_mentioned"] += 1
                
                if token.ent_type_ == "LOC":
                    info_row["n_locations_metioned"] += 1
                
                if token.ent_type_ == "PRODUCT":
                    info_row["n_products_mentioned"] += 1
            
            if not token.is_punct and not token.is_stop and token.lemma_ in common_words:
                info_row[token.lemma_] += 1

        
        info_row = dict(info_row)
        info_row["text_sentence"] = sentence
        info_row["text_source"] = source
        rows.append(info_row)
    
    if include_extra_counts:
        extra_columns = ["n_verbs", "n_nouns",
                         "n_adjectives", "n_words",
                         "n_puncutated_words", "n_right_puncutated_words",
                         "n_left_puncutated_words", "n_people_mentioned",
                         "n_locations_metioned", "n_products_mentioned"
                        ]
    else:
        extra_columns = []
    
    df = pd.DataFrame(data=rows, columns=["text_sentence"] + extra_columns + list(common_words) + ["text_source"])
    
    df = df.fillna(0)
    
    return df

In [7]:
# Group into sentences.
alice_sents = [[sent, "Carroll"] for sent in alice_doc.sents]
persuasion_sents = [[sent, "Austen"] for sent in persuasion_doc.sents]

# Combine the sentences from the two novels into one data frame.
sentences = pd.DataFrame(alice_sents + persuasion_sents)
sentences["text_sentence"] = sentences[0]
sentences["text_source"] = sentences[1]
sentences = sentences[["text_sentence", "text_source"]]
sentences.head()

,text_sentence,text_source
0,"(Alice, was, beginning, to, get, very, tired, ...",Carroll
1,"(So, she, was, considering, in, her, own, mind...",Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",Carroll
3,"(Oh, dear, !)",Carroll
4,"(I, shall, be, late, !, ')",Carroll


In [9]:
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn import ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

In [18]:
def find_best_model(models, X_train, X_test, y_train, y_test):
    
    for model, grid_parameters in models.items():
        print("Running for {}".format(str(model.__name__)))
        print("--------")
        grid = GridSearchCV(model(), grid_parameters, refit=True, verbose=0)
        
        # fit
        grid.fit(X_train, y_train)
        
        # print the score, and the best parameters
        print("Best Score: {}".format(grid.best_score_))
        print("Train Score: {}".format(grid.score(X_train, y_train)))
        print("Test Score: {}".format(grid.score(X_test, y_test)))
        print("Best Params: {}".format(grid.best_params_))
        print("")

In [10]:
# create some grid searching tools for Logistic Regression, Gradient Boosting, Random Forrest, and Support Vector Classifier

# logistic regression parameters
lr_params = {"C": [0.1, 1.0, 10., 100.], "penalty": ["l1", "l2"], "solver": ["liblinear"]}

# gradient boosting possible parameters
gb_params = {"loss": ["deviance", "exponential"], "n_estimators": [5, 10, 50, 100, 1000], "subsample": [0.1, 0.3, 0.6, 0.9, 1]}

# random forrest parameters
rdf_params = {"n_estimators": [1, 5, 10, 50, 100, 1000], "criterion": ["gini", "entropy"]}

# support vector classifier parameters
svc_params = grid_parameters = {'C': [1, 10, 100, 1000], 'gamma': [1, 0.1, 0.001, 0.0001, "scale", "auto"], 'kernel': ['linear','rbf', 'sigmoid']}

In [11]:
models_dict = {
    LogisticRegression: lr_params,
    ensemble.GradientBoostingClassifier: gb_params,
    ensemble.RandomForestClassifier: rdf_params,
    SVC: svc_params
}

In [12]:
# Set up the bags.
alicewords = bag_of_words(alice_doc)
persuasionwords = bag_of_words(persuasion_doc)

# Combine bags to create a set of unique words.
common_words = set(alicewords + persuasionwords)

In [13]:
data1 = bow_feature_generator(sentences, common_words, include_extra_counts=False)
data2 = bow_feature_generator(sentences, common_words, include_extra_counts=True)

In [14]:
data2.head()

,text_sentence,n_verbs,n_nouns,n_adjectives,n_words,n_puncutated_words,n_right_puncutated_words,n_left_puncutated_words,n_people_mentioned,n_locations_metioned,...,shrewd,vain,cheat,sake,model,turn,fan,beautifully,rational,text_source
0,"(Alice, was, beginning, to, get, very, tired, ...",6.0,10.0,1.0,67,10.0,4.0,4.0,2.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Carroll
1,"(So, she, was, considering, in, her, own, mind...",9.0,8.0,6.0,63,7.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Carroll
2,"(There, was, nothing, so, VERY, remarkable, in...",3.0,1.0,1.0,33,4.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Carroll
3,"(Oh, dear, !)",0.0,0.0,0.0,3,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Carroll
4,"(I, shall, be, late, !, ')",1.0,0.0,1.0,6,2.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,Carroll


In [15]:
features1 = data1.drop(columns=["text_source", "text_sentence"])
target1 = data1["text_source"]

features2 = data2.drop(columns=["text_source", "text_sentence"])
target2 = data2["text_source"]

In [16]:
X_train1, X_test1, y_train1, y_test1 = train_test_split(features1, target1, test_size=0.2, random_state=42)
X_train2, X_test2, y_train2, y_test2 = train_test_split(features2, target2, test_size=0.2, random_state=42)

In [19]:
find_best_model(models_dict, X_train2, X_test2, y_train2, y_test2)

Running for LogisticRegression
--------


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Score: 0.8315789473684211
Train Score: 0.9710526315789474
Test Score: 0.8421052631578947
Best Params: {'C': 1.0, 'penalty': 'l2', 'solver': 'liblinear'}

Running for GradientBoostingClassifier
--------


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Score: 0.8473684210526315
Train Score: 0.9947368421052631
Test Score: 0.8736842105263158
Best Params: {'loss': 'deviance', 'n_estimators': 1000, 'subsample': 0.1}

Running for RandomForestClassifier
--------


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Score: 0.8552631578947368
Train Score: 0.9947368421052631
Test Score: 0.9052631578947369
Best Params: {'criterion': 'entropy', 'n_estimators': 50}

Running for SVC
--------


/opt/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


Best Score: 0.85
Train Score: 0.9578947368421052
Test Score: 0.8105263157894737
Best Params: {'C': 100, 'gamma': 0.001, 'kernel': 'rbf'}

